<a href="https://www.kaggle.com/code/hsw1212/spacetitanic-ml-79?scriptVersionId=139284655" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Read csv data
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sub = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
# Fill CryoSleep, HomePlanet and Destination null data
train['VIP'] = train['VIP'].fillna(value=0)
train['VIP'] = train['VIP'].astype('int')

train['CryoSleep'] = train['CryoSleep'].fillna(value=0)
train['CryoSleep'] = train['CryoSleep'].astype('int')

train = train.replace({'HomePlanet': {'Earth': 0,
                              'Mars': 1,
                              'Europa': 2}})
train['HomePlanet'] = train['HomePlanet'].fillna(value=0)

train = train.replace({'Destination': {'TRAPPIST-1e': 0,
                              'PSO J318.5-22': 1,
                              '55 Cancri e': 2}})
train['Destination'] = train['Destination'].fillna(value=0)

train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,2.0,0,B/0/P,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,0.0,0,F/0/S,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,2.0,0,A/0/S,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,2.0,0,A/0/S,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,0.0,0,F/1/S,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,2.0,0,A/98/P,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,0.0,1,G/1499/S,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,0.0,0,G/1500/S,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,2.0,0,E/608/S,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
# Replace null data in train data (numeric data)
train['Age'].fillna(train['Age'].mean().round(0),inplace=True)
train['RoomService'].fillna(train['RoomService'].mean(),inplace=True)
train['FoodCourt'].fillna(train['FoodCourt'].mean(),inplace=True)
train['ShoppingMall'].fillna(train['ShoppingMall'].mean(),inplace=True)
train['Spa'].fillna(train['Spa'].mean(),inplace=True)
train['VRDeck'].fillna(train['VRDeck'].mean(),inplace=True)

# Drop useless columns
train.drop(columns=['PassengerId','Cabin','Name'],axis=1,inplace=True)

# Split train data and labels
y = train['Transported'].astype('int')
X = train.drop('Transported',axis=1)


In [5]:
# Processs train data
print("\n<<<<<<<<<<<<<<<<<<<< Data info >>>>>>>>>>>>>>>>>>>>\n")
train.info()
print("\n<<<<<<<<<<<<<<<<<<<< Numberr of null >>>>>>>>>>>>>>>>>>>>")
train.isnull().sum()



<<<<<<<<<<<<<<<<<<<< Data info >>>>>>>>>>>>>>>>>>>>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   float64
 1   CryoSleep     8693 non-null   int64  
 2   Destination   8693 non-null   float64
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   int64  
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  Transported   8693 non-null   bool   
dtypes: bool(1), float64(8), int64(2)
memory usage: 687.8 KB

<<<<<<<<<<<<<<<<<<<< Numberr of null >>>>>>>>>>>>>>>>>>>>


HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [6]:
# Processs test data

test['VIP'] = test['VIP'].fillna(value=0)
test['VIP'] = test['VIP'].astype('int')

test['CryoSleep'] = test['CryoSleep'].fillna(value=0)
test['CryoSleep'] = test['CryoSleep'].astype('int')

test = test.replace({'HomePlanet': {'Earth': 0,
                              'Mars': 1,
                              'Europa': 2}})
test['HomePlanet'] = test['HomePlanet'].fillna(value=0)

test = test.replace({'Destination': {'TRAPPIST-1e': 0,
                              'PSO J318.5-22': 1,
                              '55 Cancri e': 2}})
test['Destination'] = test['Destination'].fillna(value=0)


In [7]:
# Replace null data in test data (numeric data)
test['Age'].fillna(test['Age'].mean().round(0),inplace=True)
test['RoomService'].fillna(test['RoomService'].mean(),inplace=True)
test['FoodCourt'].fillna(test['FoodCourt'].mean(),inplace=True)
test['ShoppingMall'].fillna(test['ShoppingMall'].mean(),inplace=True)
test['Spa'].fillna(test['Spa'].mean(),inplace=True)
test['VRDeck'].fillna(test['VRDeck'].mean(),inplace=True)

# Drop useless columns
test.drop(columns=['PassengerId','Cabin','Name'],axis=1,inplace=True)

#
test

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,1,0.0,27.0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0.0,19.0,0,0.0,9.0,0.0,2823.0,0.0
2,2.0,1,2.0,31.0,0,0.0,0.0,0.0,0.0,0.0
3,2.0,0,0.0,38.0,0,0.0,6652.0,0.0,181.0,585.0
4,0.0,0,0.0,20.0,0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,0.0,1,0.0,34.0,0,0.0,0.0,0.0,0.0,0.0
4273,0.0,0,0.0,42.0,0,0.0,847.0,17.0,10.0,144.0
4274,1.0,1,2.0,29.0,0,0.0,0.0,0.0,0.0,0.0
4275,2.0,0,0.0,29.0,0,0.0,2680.0,0.0,0.0,523.0


In [8]:
print("\n<<<<<<<<<<<<<<<<<<<< Data info >>>>>>>>>>>>>>>>>>>>\n")
test.info()
print("\n<<<<<<<<<<<<<<<<<<<< Numberr of null >>>>>>>>>>>>>>>>>>>>")
test.isnull().sum()


<<<<<<<<<<<<<<<<<<<< Data info >>>>>>>>>>>>>>>>>>>>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   float64
 1   CryoSleep     4277 non-null   int64  
 2   Destination   4277 non-null   float64
 3   Age           4277 non-null   float64
 4   VIP           4277 non-null   int64  
 5   RoomService   4277 non-null   float64
 6   FoodCourt     4277 non-null   float64
 7   ShoppingMall  4277 non-null   float64
 8   Spa           4277 non-null   float64
 9   VRDeck        4277 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 334.3 KB

<<<<<<<<<<<<<<<<<<<< Numberr of null >>>>>>>>>>>>>>>>>>>>


HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [9]:
# Build models and use grid search approaches to find optimised model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Hyperparameter grid for Random Forest
param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Hyperparameter grid for Gradient Boosting
param_grid_gb = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.1, 1],
    'classifier__max_depth': [3, 5, 10],
}

# Hyperparameter grid for SVM
param_grid_svc = {
    'C': [0.01, 0.1, 1, 10],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf'],
}

# Hyperparameter grid for Ada Boosting
param_grid_ab = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.1, 1 ],
}

# Pipelines for the classifiers
pipeline_rf = Pipeline([
    ('classifier', RandomForestClassifier(random_state=100))
])

pipeline_gb = Pipeline([
    ('classifier', GradientBoostingClassifier(random_state=100))
])

svc_model = svm.SVC(random_state=100)

pipeline_ab = Pipeline([
    ('classifier', AdaBoostClassifier(random_state=100))
])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
# Grid search for Random Forest
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X, y)

# Grid search for Gradient Boosting
grid_search_gb = GridSearchCV(pipeline_gb, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X, y)

# Grid search for SVC
grid_search_svc = GridSearchCV(svc_model, param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svc.fit(X, y)

# Grid search for Ada Boosting
grid_search_ab = GridSearchCV(pipeline_ab, param_grid_ab, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_ab.fit(X, y)

# Best models and scores
best_model_rf, best_score_rf = grid_search_rf.best_estimator_, grid_search_rf.best_score_
best_model_gb, best_score_gb = grid_search_gb.best_estimator_, grid_search_gb.best_score_
best_model_svc, best_score_svc = grid_search_svc.best_estimator_, grid_search_svc.best_score_
best_model_ab, best_score_ab = grid_search_ab.best_estimator_, grid_search_ab.best_score_

best_model_rf, best_score_rf, best_model_gb, best_score_gb,best_model_svc, best_score_svc, best_model_ab, best_score_ab

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(Pipeline(steps=[('classifier',
                  RandomForestClassifier(max_depth=10, min_samples_leaf=2,
                                         min_samples_split=10, n_estimators=300,
                                         random_state=100))]),
 0.7976560209794791,
 Pipeline(steps=[('classifier',
                  GradientBoostingClassifier(n_estimators=200,
                                             random_state=100))]),
 0.7947791510140017,
 SVC(C=1, gamma=0.001, random_state=100),
 0.7440494947362708,
 Pipeline(steps=[('classifier',
                  AdaBoostClassifier(learning_rate=1, n_estimators=100,
                                     random_state=100))]),
 0.7890277271370726)

In [11]:
# Do prediction
pred = best_model_rf.predict(test)

# pred.astype('bool')

output = pd.DataFrame({'PassengerId': sub.PassengerId, 'Transported': pred.astype('bool')})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
